In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [2]:
class Graph():
    def __init__(self, layout='openpose', strategy='spatial'):
        self.get_edge(layout)
        self.hop_dis = self.get_hop_distance(self.num_node, self.edge)
        self.get_adjacency(strategy)

    def get_edge(self, layout):
        if layout == 'openpose':
            self.num_node = 18  # or whatever number matches your data
            print(f"Graph: num_node = {self.num_node}")
            self_link = [(i, i) for i in range(self.num_node)]
            neighbor_link = [(4, 3), (3, 2), (7, 6), (6, 5), (13, 12), (12, 11),
                             (10, 9), (9, 8), (11, 5), (8, 2), (5, 1), (2, 1),
                             (0, 1), (15, 0), (14, 0), (17, 15), (16, 14)]
            self.edge = self_link + neighbor_link
            self.center = 1
        else:
            raise ValueError("Do not support this layout.")

    def get_hop_distance(self, num_node, edge, max_hop=1):
        A = np.zeros((num_node, num_node))
        for i, j in edge:
            A[j, i] = 1
            A[i, j] = 1

        hop_dis = np.zeros((num_node, num_node)) + np.inf
        transfer_mat = [np.linalg.matrix_power(A, d) for d in range(max_hop + 1)]
        arrive_mat = (np.stack(transfer_mat) > 0)
        for d in range(max_hop, -1, -1):
            hop_dis[arrive_mat[d]] = d
        return hop_dis

    def get_adjacency(self, strategy):
        valid_hop = range(0, self.num_node - 1)
        adjacency = np.zeros((self.num_node, self.num_node))
        for hop in valid_hop:
            adjacency[self.hop_dis == hop] = 1
        normalize_adjacency = self.normalize_digraph(adjacency)

        if strategy == 'uniform':
            A = np.zeros((1, self.num_node, self.num_node))
            A[0] = normalize_adjacency
            self.A = A
        elif strategy == 'distance':
            A = np.zeros((len(valid_hop), self.num_node, self.num_node))
            for i, hop in enumerate(valid_hop):
                A[i][self.hop_dis == hop] = normalize_adjacency[self.hop_dis == hop]
            self.A = A
        elif strategy == 'spatial':
            A = []
            for hop in valid_hop:
                a_root = np.zeros((self.num_node, self.num_node))
                a_close = np.zeros((self.num_node, self.num_node))
                a_further = np.zeros((self.num_node, self.num_node))
                for i in range(self.num_node):
                    for j in range(self.num_node):
                        if self.hop_dis[j, i] == hop:
                            if self.hop_dis[j, self.center] == self.hop_dis[i, self.center]:
                                a_root[j, i] = normalize_adjacency[j, i]
                            elif self.hop_dis[j, self.center] > self.hop_dis[i, self.center]:
                                a_close[j, i] = normalize_adjacency[j, i]
                            else:
                                a_further[j, i] = normalize_adjacency[j, i]
                if hop == 0:
                    A.append(a_root)
                else:
                    A.append(a_root + a_close)
                    A.append(a_further)
            A = np.stack(A)
            self.A = A
        else:
            raise ValueError("Do not support this strategy.")

    def normalize_digraph(self, A):
        Dl = np.sum(A, 0)
        num_node = A.shape[0]
        Dn = np.zeros((num_node, num_node))
        for i in range(num_node):
            if Dl[i] > 0:
                Dn[i, i] = Dl[i]**(-1)
        AD = np.dot(A, Dn)
        return AD

In [3]:
class ConvTemporalGraphical(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, t_kernel_size=1, t_stride=1, t_padding=0, t_dilation=1, bias=True):
        super().__init__()
        self.kernel_size = kernel_size
        self.conv = nn.Conv2d(in_channels, out_channels * kernel_size, kernel_size=(t_kernel_size, 1), 
                              padding=(t_padding, 0), stride=(t_stride, 1), dilation=(t_dilation, 1), bias=bias)

    def forward(self, x, A):
        assert A.size(0) == self.kernel_size
        x = self.conv(x)
        n, kc, t, v = x.size()
        x = x.view(n, self.kernel_size, kc//self.kernel_size, t, v)
        x = torch.einsum('nkctv,kvw->nctw', (x, A))
        return x.contiguous(), A



In [4]:
class STGCNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dropout=0, residual=True):
        super().__init__()
        assert len(kernel_size) == 2
        assert kernel_size[0] % 2 == 1
        padding = ((kernel_size[0] - 1) // 2, 0)
        
        self.gcn = ConvTemporalGraphical(in_channels, out_channels, kernel_size[1])
        self.tcn = nn.Sequential(
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, (kernel_size[0], 1), (stride, 1), padding),
            nn.BatchNorm2d(out_channels),
            nn.Dropout(dropout, inplace=True)
        )
        
        if not residual:
            self.residual = lambda x: 0
        elif (in_channels == out_channels) and (stride == 1):
            self.residual = lambda x: x
        else:
            self.residual = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=(stride, 1)),
                nn.BatchNorm2d(out_channels)
            )
        
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x, A):
        res = self.residual(x)
        x, A = self.gcn(x, A)
        x = self.tcn(x) + res
        return self.relu(x), A

In [5]:
class STGCN(nn.Module):
    def __init__(self, in_channels, graph_args, edge_importance_weighting, **kwargs):
        super().__init__()
        self.graph = Graph(**graph_args)
        A = torch.tensor(self.graph.A, dtype=torch.float32, requires_grad=False)
        self.register_buffer('A', A)
        
        spatial_kernel_size = A.size(0)
        temporal_kernel_size = 9
        kernel_size = (temporal_kernel_size, spatial_kernel_size)
        
        # Replace BatchNorm with GroupNorm
        self.data_bn = nn.GroupNorm(num_groups=4, num_channels=in_channels * A.size(1))
        
        self.st_gcn_networks = nn.ModuleList((
            STGCNBlock(in_channels, 64, kernel_size, stride=1, residual=False),
            STGCNBlock(64, 64, kernel_size, stride=1),
            STGCNBlock(64, 64, kernel_size, stride=1),
            STGCNBlock(64, 64, kernel_size, stride=1),
            STGCNBlock(64, 128, kernel_size, stride=2),
            STGCNBlock(128, 128, kernel_size, stride=1),
            STGCNBlock(128, 128, kernel_size, stride=1),
            STGCNBlock(128, 256, kernel_size, stride=2),
            STGCNBlock(256, 256, kernel_size, stride=1),
            STGCNBlock(256, 256, kernel_size, stride=1)
        ))
        
        if edge_importance_weighting:
            self.edge_importance = nn.ParameterList([
                nn.Parameter(torch.ones(self.A.size()))
                for i in self.st_gcn_networks
            ])
        else:
            self.edge_importance = [1] * len(self.st_gcn_networks)
    
    def forward(self, x):
        N, C, T, V = x.size()
        x = x.permute(0, 3, 1, 2).contiguous()
        x = x.view(N, V * C, T)
        x = self.data_bn(x)
        x = x.view(N, V, C, T)
        x = x.permute(0, 2, 3, 1).contiguous()
        x = x.view(N, C, T, V)
        
        for gcn, importance in zip(self.st_gcn_networks, self.edge_importance):
            x, _ = gcn(x, self.A * importance)
        
        x = F.avg_pool2d(x, x.size()[2:])
        x = x.view(N, -1)
        
        return x



In [6]:
class ParkinsonsDetectionModel(nn.Module):
    def __init__(self, in_channels, graph_args, edge_importance_weighting, **kwargs):
        super().__init__()
        self.st_gcn = STGCN(in_channels, graph_args, edge_importance_weighting, **kwargs)
        self.angle_embedding = nn.Embedding(3, 64)  # 3 angles (0, 90, 180), embedding size 64
        self.fc1 = nn.Linear(256 + 64, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)  # Binary classification
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, angle):
        x = self.st_gcn(x)
        angle = torch.div(angle, 90, rounding_mode='floor').long()  # Convert angles to 0, 1, 2
        angle_embed = self.angle_embedding(angle)
        combined = torch.cat([x, angle_embed], dim=1)
        x = F.relu(self.fc1(combined))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x.squeeze(-1)  # Remove the last dimension if it's 1

In [7]:
# Example usage
graph_args = {'layout': 'openpose', 'strategy': 'spatial'}
model = ParkinsonsDetectionModel(in_channels=2, graph_args=graph_args, edge_importance_weighting=True)

# Example input
batch_size = 2
num_channels = 2  # X and Y coordinates
num_frames = 300
num_joints = 18
x = torch.randn(batch_size, num_channels, num_frames, num_joints)
angle = torch.LongTensor([0, 1])  # Example angles for each sample in the batch

# Forward pass
output = model(x, angle)
print(output.shape)  # Should be [batch_size, 1]

Graph: num_node = 18
torch.Size([2])


In [8]:
angle

tensor([0, 1])

In [9]:
print(x)

tensor([[[[-2.0335,  1.4941, -0.2940,  ...,  0.9594, -0.1077,  1.4888],
          [ 0.3492, -0.2053, -0.1154,  ...,  0.2667, -0.9089, -0.4013],
          [-1.4240,  0.1997, -1.2834,  ...,  0.6216, -0.3103, -0.2079],
          ...,
          [ 0.3061, -0.5555,  0.5692,  ..., -0.7460,  2.8792,  0.8326],
          [ 0.7999, -0.3021,  0.5384,  ..., -0.3186, -0.5753, -0.5013],
          [-0.4910,  0.5218,  0.4255,  ...,  0.2451, -1.2994, -0.4517]],

         [[ 1.5659, -0.3193,  0.5577,  ..., -0.5505,  1.7439, -2.0961],
          [ 1.3964, -0.2915,  0.0970,  ..., -0.2505, -2.0654, -0.2219],
          [ 0.0689, -0.3233, -2.5307,  ...,  0.9248,  0.5405,  0.2090],
          ...,
          [ 1.7750, -0.4143, -1.4958,  ...,  0.6128, -0.1692,  1.5159],
          [-1.4624, -0.9879,  0.8400,  ...,  1.7351, -0.6053, -2.1655],
          [-0.1979, -1.6130, -1.5611,  ..., -1.1263,  1.0781, -0.0283]]],


        [[[ 2.4037,  0.6105,  0.4014,  ...,  0.9103,  0.7881,  0.4280],
          [ 1.5694, -0.3508,

In [10]:
x[0][0][0]

tensor([-2.0335,  1.4941, -0.2940,  0.5986, -0.5813, -0.3524,  0.2723, -0.7479,
        -0.0283, -0.3284,  0.5232, -0.6634,  0.1028, -1.2395, -1.8757,  0.9594,
        -0.1077,  1.4888])

In [11]:
# Load the CSV file
df = pd.read_csv("../output_data/output_2D.csv")

# Convert the vector column from string to numpy array
df['vector'] = df['vector'].apply(eval).apply(np.array)


In [12]:
df

,client,angle,parkinson,vector
0,1,0,1,"[[38.89351272583008, 17.675010681152344], [43...."
1,1,0,1,"[[85.9338607788086, 29.34123420715332], [92.69..."
2,1,0,1,"[[50.33542251586914, 20.45751190185547], [55.2..."
3,1,0,1,"[[44.83964920043945, 21.37966537475586], [50.3..."
4,1,0,1,"[[64.98465728759766, 29.381858825683594], [70...."
...,...,...,...,...
546,1,90,1,"[[157.67861938476562, 44.677371978759766], [15..."
547,1,90,1,"[[104.82242584228516, 46.87606430053711], [109..."
548,1,90,1,"[[135.35787963867188, 49.76270294189453], [138..."
549,1,90,1,"[[133.68704223632812, 43.090797424316406], [13..."


In [13]:
#get max length of the vectors
max_length = df['vector'].apply(len).max()

# Pad the vectors to the max length
df['vector'] = df['vector'].apply(lambda x: np.pad(x, ((0, max_length - len(x)), (0, 0))))

In [14]:
df

,client,angle,parkinson,vector
0,1,0,1,"[[38.89351272583008, 17.675010681152344], [43...."
1,1,0,1,"[[85.9338607788086, 29.34123420715332], [92.69..."
2,1,0,1,"[[50.33542251586914, 20.45751190185547], [55.2..."
3,1,0,1,"[[44.83964920043945, 21.37966537475586], [50.3..."
4,1,0,1,"[[64.98465728759766, 29.381858825683594], [70...."
...,...,...,...,...
546,1,90,1,"[[157.67861938476562, 44.677371978759766], [15..."
547,1,90,1,"[[104.82242584228516, 46.87606430053711], [109..."
548,1,90,1,"[[135.35787963867188, 49.76270294189453], [138..."
549,1,90,1,"[[133.68704223632812, 43.090797424316406], [13..."


In [15]:
def parse_vector(vector):
    vector_array = np.array(vector)
    # Ensure we have exactly 34 joints with 2 coordinates each
    if vector_array.shape[0] != 18 or vector_array.shape[1] != 2:
        padded = np.zeros((18, 2))
        padded[:min(vector_array.shape[0], 18), :min(vector_array.shape[1], 2)] = vector_array[:18, :2]
        return padded
    return vector_array



In [16]:
# Parse and stack the pose data
pose_data = np.stack(df['vector'].apply(parse_vector).values)

# Reshape to [N, C, T, V]
N, V, C = pose_data.shape
T = 1  # Assuming each sample is a single frame
pose_data = pose_data.transpose(0, 2, 1).reshape(N, C, T, V)

print("Pose data shape after reshaping:", pose_data.shape)

# Convert to torch tensors
pose_data_tensor = torch.FloatTensor(pose_data)
angles_tensor = torch.LongTensor(df['angle'].values)
labels_tensor = torch.FloatTensor(df['parkinson'].values)

print("Final pose data tensor shape:", pose_data_tensor.shape)

Pose data shape after reshaping: (551, 2, 1, 18)
Final pose data tensor shape: torch.Size([551, 2, 1, 18])


In [17]:
print("Pose data shape:", pose_data_tensor.shape)

Pose data shape: torch.Size([551, 2, 1, 18])


In [18]:
pose_data_tensor[0][0][0]

tensor([38.8935, 43.8592, 35.0178, 51.7123, 29.7362, 62.4427, 19.7756, 76.1154,
         5.2351, 76.3991,  4.9302, 54.7671, 27.9492, 55.6615, 27.8608, 55.0021,
        31.3362,  0.0000])

In [19]:
graph_args = {'layout': 'openpose', 'strategy': 'spatial'}
in_channels = pose_data_tensor.shape[1]
model = ParkinsonsDetectionModel(in_channels=in_channels, graph_args=graph_args, edge_importance_weighting=True)


Graph: num_node = 18


In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
pose_data_tensor = pose_data_tensor.to(device)
angles_tensor = angles_tensor.to(device)
labels_tensor = labels_tensor.to(device)
criterion = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss instead of BCELoss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 3
batch_size = 10  # or whatever size works for your memory constraints


for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for i in range(0, pose_data_tensor.size(0), batch_size):
        # Get batch
        batch_pose = pose_data_tensor[i:i+batch_size]
        batch_angles = angles_tensor[i:i+batch_size]
        batch_labels = labels_tensor[i:i+batch_size]

        # Forward pass
        outputs = model(batch_pose, batch_angles)
        loss = criterion(outputs, batch_labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / (pose_data_tensor.size(0) // batch_size)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/3], Loss: 0.0625
Epoch [2/3], Loss: 0.0001
Epoch [3/3], Loss: 0.0000


In [22]:
# For inference:
model.eval()
with torch.no_grad():
    outputs = model(pose_data_tensor, angles_tensor)
    predicted = (outputs.squeeze() > 0.5).float()  # Threshold at 0.5 for binary classification
    accuracy = (predicted == labels_tensor).float().mean()
    print("Accuracy:", accuracy.item())

Accuracy: 1.0
